In [38]:
import pandas as pd
from xml.etree import ElementTree
from xml.etree.ElementTree import Element

import os


MERCANTE = 'mercante'

In [39]:
lista_arquivos = os.listdir(MERCANTE)

In [40]:
with open(os.path.join(MERCANTE, lista_arquivos[0])) as file0:
    content0 = file0.read()

In [41]:
xtree = ElementTree.parse(os.path.join(MERCANTE, lista_arquivos[0]))
xroot = xtree.getroot()

In [65]:
print(xroot)
for node in xroot:
    print(node)
    print(node.tag)
    if isinstance(node, Element):
        for subnode in node:
            print(subnode.tag)
            print(subnode.text)


<Element 'cargas' at 0x000002CCB9775B38>
<Element 'conhecimentosEmbarque' at 0x000002CCB9775A98>
conhecimentosEmbarque
tipoMovimento
A
dataAtualizacao
2019-09-26
horaAtualizacao
23:55:19
numeroCEmercante
131905212301619
tipoTrafego
5
numConhecimento
TSYD011281
tipoBLConhecimento
11
quantidadeFilhotes
1
cnpjAgenteDesconsolidador
03094658000106
empresaNVOCC
CN004522
dataEmissao
2019-08-20
cubagem
0000000076.000
portoDestFinal
BRRIO
portoOrigemCarga
CNTXG
descricao
 52 PK (PACKAGE(S)).ALLOY WHEELINV NUMBER : DC00514C17109/DC00514C17109-1NCM: 8708.70.90PACKAGING : PROCESSED WOODFREIGHT PREPAID40HCX2*TEL:86 22 58853790FAX:86 22 5885379342ALLOY WHEELINV NUMBER : DC00514C17109/DC00514C17109-1NCM: 8708.70.90PACKAGING : PROCESSED WOODFREIGHT PREPAID40HCX2*TEL:86 22 58853790FAX:86 22 5885379310
embarcador


indicadorBLServico
false
indicadorBLOrdem
false
indicadorConsignatarioEstrangeiro
false
idEntrega
false
frete


consignatario


entregaExterior


codigoTerminalDescarregamento
BRRIO002
paisPr

In [136]:
class Conhecimento:
    numeroCEmercante:str = None
    consignatario:str = None

    
    def _campos(self):
        return [campo for campo in dir(Conhecimento) if campo[0] != '_']
        
    def __repr__(self):
        repr = ' '.join([ campo + '-' + getattr(self, campo) for campo in self._campos()])
        if repr is None:
            return 'ERRO'
        return repr
        

In [137]:
Conhecimento()._campos()

['consignatario', 'numeroCEmercante']

In [138]:

def parse_node(subnode, objeto):
    for campo in objeto._campos():
        print(campo)
        print(subnode.find(campo).text)
        setattr(objeto, campo, subnode.find(campo).text)
    
    
for node in xroot:
    if node.tag == 'conhecimentosEmbarque':
        objeto = Conhecimento()
    parse_node(node, objeto)
    print(objeto)

consignatario


numeroCEmercante
131905212301619
consignatario-
 numeroCEmercante-131905212301619
consignatario


numeroCEmercante
131905215824167
consignatario-
 numeroCEmercante-131905215824167


In [139]:
[(campo, getattr(objeto, campo)) for campo in objeto._campos()]

[('consignatario', '\n'), ('numeroCEmercante', '131905215824167')]

In [131]:
' '.join([ campo + '-' + getattr(objeto, campo) for campo in objeto._campos()])

'consignatario-\n numeroCEmercante-131905212301619'